In [2]:
import requests
from bs4 import BeautifulSoup
import json

/Users/Welcome/Library/Python/3.9/lib/python/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [ ]:
#utils.py part
#type에 맞지 않는 데이터 
TYPE_CONFIG = {
    "사회": "society",
    "정치": "politics",
    "경제": "economic",
    "국제": "foreign",
    "문화": "culture",
    "연예": "entertain",
    "스포츠": "sports",
    "IT": "digital",
    "칼럼": "editorial",
    "보도자료": "press",
}

In [31]:
#**kwars 이거ㅇㅇ
def makeURL(search = "", period="", start_date="", end_date="") -> str:
    url = "https://search.daum.net/search?w=news&DA=STC&enc=utf8"
    period_url = ""
    search_url = ""
    
    if period != "":
        period_url = "&period=%s" % (period)
        
        if period == "u": #기간 조정
            period_url = period_url + "&sd=%s&ed=%s" % (start_date, end_date)
    
    if search !="":
        search_url = "&q=%s" % (search)
    
    return url + period_url + search_url

def newsScrape(search):
    URL = makeURL(search)
    original_html = requests.get(URL)
    html = BeautifulSoup(original_html.text, "html.parser")
    title_list = html.select("div.cont_divider > ul.list_news > li div.wrap_cont > a")
    #content_list = html.select("div.cont_divider > ul.list_news > li div.wrap_cont > p")

    for i in title_list:
        print(i.get_text())
        
def newsType(newsType=""):
    URL = "https://news.daum.net/breakingnews/"
    if newsType !="":
        URL = URL + newsType
    
    original_html = requests.get(url)
    html = BeautifulSoup(original_html.text, "html.parser")
    title_list = html.select("div.box_etc > ul.list_news2.list_allnews > li div.cont_thumb > strong.tit_thumb > a")
    
    for i in title_list:
        print(i.get_text())



#newsScrape("우영우")

#search 없을때 날짜로만 데이터 수집할때는




IPA, 하반기 체험형 청년인턴 채용..8월3일까지 원서 접수
클럽·자택서 마약 투약·흡입 20~30대 5명 검찰 송치
[속보] 코로나 예방접종자 심근경색·뇌경색 위험 58% '감소'..25만명 연구
[속보] 백신 2차접종 후 확진자, 심근경색·뇌졸중 발생 절반 이하
청주국제공항, 베트남·몽골 부정기 국제선 운항 재개
충청북도해양교육원 제주분원 곽지 청춘별빛음악회 개최
[속보] 면역회피 강하고 전파 빠른 'BA.5' 사실상 우세종..검출률 56.3%
'달빛 항해 희망을 향해' 청주문화재야행 8월27~28일 개최
충북자연과학교육원 충북형 노벨 과학프로젝트 개최
영동학산 농협 상호금융대상평가 전국그룹별 1위
강원소방, 소방차 44대 성능평가..교체 여부 결정
괴산군, '제14회 아름다운 괴산 관광사진 전국공모전' 수상작 선정
대한적십자사봉사회 괴산군지구협의회, '후원금' 전달
괴산 육군학생군사학교, 저소득 독거노인을 위한 따뜻한 기부
음성군농업기술센터, '바른 식문화' 교육 성료
